In [1]:
# Assignment Page 142: 11-1
## Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth.
## Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? 
## You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.import scipy.stats
import numpy as np
import random
import thinkstats2
import thinkplot

In [2]:
from os.path import basename, exists
def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)

In [5]:
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/nsfg.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/first.py")

download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dct")
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dat.gz"
)

In [6]:
import patsy
# Go for data mining to search the variables with explanatory power
def GoMining(df):
# Searches for variables that predict pregnancy length. df: DataFrame of pregnancy records
##. returns: list of (rsquared, variable name) pairs

    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula = 'prglngth ~ agepreg + ' + name
            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:
                continue

            results = model.fit()
        except (ValueError, TypeError, patsy.PatsyError) as e:
            continue
        
        variables.append((results.rsquared, name))

    return variables

In [9]:
# define dataframe for 30th week of pregnancy
import first
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]

In [10]:
live.head(5)

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,laborfor_i,religion_i,metro_i,basewgt,adj_mod_basewgt,finalwgt,secu_p,sest,cmintvw,totalwgt_lb
0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,8.8125
1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,7.8750
2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,5.0,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,9.1250
3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,7.0000
4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,6.1875


In [13]:
# use join to combine variables from the preganancy and respondent tables.
import nsfg
import statsmodels.formula.api as smf
live = live[live.prglngth>30]
resp = nsfg.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')
join.shape

(8884, 3331)

In [15]:
# call GoMining function to findout variables with explanatory power
variables = GoMining(join)
variables

[(0.005357647323640635, 'caseid'),
 (0.005750013985077129, 'pregordr'),
 (0.006330980237390427, 'pregend1'),
 (0.016017752709788224, 'nbrnaliv'),
 (0.005543156193094756, 'cmprgend'),
 (0.005442800591639707, 'cmprgbeg'),
 (0.005327612601561005, 'gestasun_m'),
 (0.007023552638453112, 'gestasun_w'),
 (0.12340041363361076, 'wksgest'),
 (0.027144274639580024, 'mosgest'),
 (0.005336869167517633, 'bpa_bdscheck1'),
 (0.018550925293941978, 'babysex'),
 (0.9498127305978009, 'birthwgt_lb'),
 (0.013102457615706053, 'birthwgt_oz'),
 (0.005543156193094756, 'cmbabdob'),
 (0.005684952650028108, 'kidage'),
 (0.006165319836040184, 'hpagelb'),
 (0.0080663173686768, 'matchfound'),
 (0.012529022541810764, 'anynurse'),
 (0.004409820583625823, 'frsteatd_n'),
 (0.00426397347170937, 'frsteatd_p'),
 (0.004020131462736054, 'frsteatd'),
 (0.005830571770254145, 'cmlastlb'),
 (0.005356747266123563, 'cmfstprg'),
 (0.005428333650990047, 'cmlstprg'),
 (0.0057314017337595224, 'cmintstr'),
 (0.005543156193094756, 'cmint

In [18]:
# The following functions report the variables with the highest values of  𝑅2
import re
import pandas as pd
def ReadVariables():
# Reads Stata dictionary files for NSFG data. returns: DataFrame that maps variables names to descriptions

    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

    all_vars = vars1.append(vars2)
    all_vars.index = all_vars.name
    return all_vars

def MiningReport(variables, n=30):
    """Prints variables with the highest R^2.

    t: list of (R^2, variable name) pairs
    n: number of pairs to print
    """
    all_vars = ReadVariables()

    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            print(name, r2, desc)
        except (KeyError, IndexError):
            print(name, r2)

In [19]:
MiningReport(variables)

totalwgt_lb 1.0
birthwgt_lb 0.9498127305978009 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.30082407844707704 LOW BIRTHWEIGHT - BABY 1
prglngth 0.13012519488625063 DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest 0.12340041363361076 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
agecon 0.10203149928156086 AGE AT TIME OF CONCEPTION
mosgest 0.027144274639580024 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
babysex 0.018550925293941978 BD-2 SEX OF 1ST LIVEBORN BABY FROM THIS PREGNANCY
race_r 0.016199503586252995 RACE
race 0.016199503586252995 RACE
nbrnaliv 0.016017752709788224 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
paydu 0.014003795578114597 IB-10 CURRENT LIVING QUARTERS OWNED/RENTED, ETC
rmarout03 0.01343006646571343 INFORMAL MARITAL STATUS WHEN PREGNANCY ENDED - 3RD
birthwgt_oz 0.013102457615706053 BD-3 BIRTHWEIGHT IN OUNCES - 1ST BABY FROM THIS PREGNANCY
anynurse 0.012529022541810764 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM T

/var/folders/s5/r__yh4p17f12tlyfs6w_46xm0000gn/T/ipykernel_38378/114085406.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_vars = vars1.append(vars2)


In [20]:
# Ans--> Combining the variables that seem to have the most explanatory power and variables that are known before the birth
#     The following are the variables that have a statistically significant effect on pregnancy length.
##    a. birthord b. race(2 white) c. nbrnaliv  (UMBER OF BABIES BORN ALIVE) d.agepreg with R square 0.011

import statsmodels.formula.api as smf
model = smf.ols('prglngth ~ birthord==1 + race==2 + nbrnaliv>1 + agepreg', data=live)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     25.72
Date:                Sun, 14 May 2023   Prob (F-statistic):           3.16e-21
Time:                        20:49:04   Log-Likelihood:                -18247.
No. Observations:                8884   AIC:                         3.650e+04
Df Residuals:                    8879   BIC:                         3.654e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                38.7357      0.105    367.466      0.000      38.529      38.942
birthord == 1[T.True]     0.1052      0.043      2.473      0.013       0.022       0.189
race == 2[T.True]         0.1369      0.043      3.202      0.001       0.053       0.221
nbrnaliv > 1[T.True]     -1.4956      0.165     -9.089      0.000      -1.818      -1.173
agepreg                   0.0010      0.004      0.264      0.792      -0.007       0.009
==============================================================================
Omnibus:                     1587.420   Durbin-Watson:                   1.620
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6159.875
Skew:                          -0.852   Prob(JB):                         0.00
Kurtosis:                       6.707   Cond. No.                         210.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# Assignment Page 142: 11-3 
## The Trivers-Willard hypothesis suggests that for many mammals the sex ratio depends on “maternal condition”; that is, factors like the mother’s age, size, health, and social status. See https://en.wikipedia.org/wiki/Trivers-Willard_hypothesis
## Some studies have shown this effect among humans, but results are mixed. In this chapter we tested some variables related to these factors, but didn’t find any with a statistically significant effect on sex ratio.
## As an exercise, use a data mining approach to test the other variables in the pregnancy and respondent files. Can you find any factors with a substantial effect?


In [21]:
# Define the go mining function to searches for variables that predict sex ( boy )
# df: DataFrame of pregnancy records ,returns: list of (rsquared, variable name) pairs

def GoMining(df):
    df['boy'] = (df.babysex==1).astype(int)
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula='boy ~ agepreg + ' + name
            model = smf.logit(formula, data=df)
            nobs = len(model.endog)
            if nobs < len(df)/2:
                continue

            results = model.fit()
        except:
            continue

        variables.append((results.prsquared, name))

    return variables

variables = GoMining(join)

Optimization terminated successfully.
         Current function value: 0.692991
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692961
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692849
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692996
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692903
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692724
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.693014
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692992
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693010
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692985
  

/Users/rajibsamanta/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 0.693003
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692998
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693015
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692973
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692973
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692810
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693014
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693015
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693011
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693003
  

/Users/rajibsamanta/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 0.692726
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692774
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692999
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692861
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692705
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692723
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692803
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692956
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692786
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693007
  

...

Optimization terminated successfully.
         Current function value: 0.692855
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692855
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693013
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692749
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692862
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692862
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692821
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692772
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692687
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692947
  

/Users/rajibsamanta/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/rajibsamanta/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 0.692957
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693015
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692983
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692258
         Iterations 5
         Current function value: 0.692696
         Iterations: 35
Optimization terminated successfully.
         Current function value: 0.692993
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693009
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693011
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692853
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692971
         Iterations 3
Optimization ter

/Users/rajibsamanta/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 0.692815
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693002
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693011
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692989
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693008
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693011
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693011
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693011
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693011
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693011
  

In [22]:
# MiningReport report the variables with the highest values of  𝑅2
MiningReport(variables)

totalwgt_lb 0.009696855253233383
birthwgt_lb 0.009274460080281988 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
constat3 0.0010985419170438382 3RD PRIORITY CODE FOR CURRENT CONTRACEPTIVE STATUS
lbw1 0.0010519527860076705 LOW BIRTHWEIGHT - BABY 1
nplaced 0.001010368752280555 # OF R'S BIO CHILDREN SHE PLACED FOR ADOPTION (BASED ON BPA)
fmarout5 0.0009096579032891183 FORMAL MARITAL STATUS AT PREGNANCY OUTCOME
rmarout6 0.000818252143711895 INFORMAL MARITAL STATUS AT PREGNANCY OUTCOME - 6 CATEGORIES
infever 0.0008115919859909004 EVER USED INFERTILITY SERVICES OF ANY KIND
frsteatd 0.0007675331422082321 AGE (IN MOS) WHEN 1ST SUPPLEMENTED - 1ST FROM THIS PREG
splstwk1 0.0007334122339932581 IF-1 H/P DOING WHAT LAST WEEK (EMPLOYMENT STATUS) 1ST MENTION
pmarpreg 0.0007245809157658822 WHETHER PREGNANCY ENDED BEFORE R'S 1ST MARRIAGE (PREMARITALLY)
usefstp 0.0007122387685902787 EF-3 USE METHOD AT FIRST SEX WITH 1ST PARTNER IN PAST 12 MONTHS?
outcom02 0.0007015744602576479 OUTCOME OF PREG

/var/folders/s5/r__yh4p17f12tlyfs6w_46xm0000gn/T/ipykernel_38378/114085406.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_vars = vars1.append(vars2)


In [38]:
# Ans--> Eliminating variables that are not known during pregnancy and others that are fishy for various reasons, here's the best model I could find:
# The model value is 0.691874. There are few varaibles are has impact which is also less . below variables have p value 0.001	
# agepreg --> Age at pregnancy outcome
# fmarout5 --> Formal marital status at pregnancy outcome , 5 NEVER MARRIE--p value 0.006 , means significant
# infever --> EVER USED INFERTILITY SERVICES OF ANY KIND --p value 0.001 , means significant
#  RMARITAL -->Informal Marital Status, CURRENTLY MARRIED =1, P value 0.671 not significant 
formula = 'boy ~ agepreg + fmarout5==5 + infever==1 + rmarital==1'
model = smf.logit(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 0.691864
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    boy   No. Observations:                 8884
Model:                          Logit   Df Residuals:                     8879
Method:                           MLE   Df Model:                            4
Date:                Mon, 15 May 2023   Pseudo R-squ.:                0.001668
Time:                        00:14:50   Log-Likelihood:                -6146.5
converged:                       True   LL-Null:                       -6156.8
Covariance Type:            nonrobust   LLR p-value:                 0.0003913
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                -0.1690      0.121     -1.399      0.162      -0.406       0.068
fmarout5 == 5[T.True]     0.1489      0.054      2.768      0.006       0.043       0.254
infever == 1[T.True]      0.2204      0.065      3.387      0.001       0.093       0.348
rmarital == 1[T.True]    -0.0208      0.049     -0.425      0.671      -0.117       0.075
agepreg                   0.0052      0.004      1.202      0.229      -0.003       0.014
=========================================================================================
"""

In [ ]:
# Assignment Page 143: 11-4: 
# If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called mnlogit. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called rmarital.
# Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [ ]:
# Ans--> To find out the outcome rmarital( Marital Status) independent variables are
# a. age_r --> Age   b. race--> Race (2 WHITE) c. totincr--> income (11 $40,000-$49,999) d. educat-->Education (12 12TH GRADE)


In [36]:
# Build the formaul with above variables

formula='rmarital ~ age_r +  C(race) + totincr + educat'
model = smf.mnlogit(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.087603
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8854
Method:                           MLE   Df Model:                           25
Date:                Sun, 14 May 2023   Pseudo R-squ.:                  0.1655
Time:                        21:44:03   Log-Likelihood:                -9662.3
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        4.4532      0.279     15.977      0.000       3.907       5.000
C(race)[T.2]    -0.9219      0.089    -10.409      0.000      -1.095      -0.748
C(race)[T.3]    -0.6334      0.136     -4.674      0.000      -0.899      -0.368
age_r           -0.0570      0.006     -9.754      0.000      -0.068      -0.046
totincr         -0.1302      0.012    -11.298      0.000      -0.153      -0.108
educat          -0.2051      0.019    -11.017      0.000      -0.242      -0.169
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -4.5432      0.916     -4.960      0.000      -6.338      -2.748
C(race)[T.2]    -0.4405      0.236     -1.865      0.062      -0.904       0.023
C(race)[T.3]     0.0329      0.335      0.098      0.922      -0.623       0.689
age_r            0.1301      0.019      6.833      0.000       0.093       0.167
totincr         -0.3251      0.032    -10.182      0.000      -0.388      -0.263
educat          -0.1010      0.048     -2.100      0.036      -0.195      -0.007
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.6485      0.302     -8.783      0.000      -3.239      -2.057
C(race)[T.2]    -0.3188      0.093     -3.417      0.001      -0.502      -0.136
C(race)[T.3]    -0.7664      0.171     -4.485      0.000      -1.101      -0.431
age_r            0.0641      0.007      9.817      0.000       0.051       0.077
totincr         -0.2270      0.012    -19.588      0.000      -0.250      -0.204
educat           0.0673      0.017      4.036      0.000       0.035       0.100
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.8231      0.345      2.385      0.017       0.147       1.500
C(race)[T.2]    -1.0491      0.104    -10.125      0.000      -1.252      -0.846
C(race)[T.3]    -0.5644      0.156     -3.624      0.000      -0.870      -0.259
age_r            0.0063      0.007      0.872      0.383      -0.008       0.020
totincr         -0.2923      0.015    -20.149      0.000      -0.321      -0.264
educat          -0.0136      0.021     -0.639      0.523      -0.056       0.028
--------------------------------------------------------------------------------
  rmarital=6       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        6.2370      0.275     22.654      0.000       5.697       6.777
C

In [37]:
# Now run the predict function
columns = ['age_r', 'race', 'totincr', 'educat']
new = pd.DataFrame([[25, 2, 11, 12]], columns=columns)
results.predict(new)

#Value Label Total
#1 CURRENTLY MARRIED 
#2 NOT MARRIED BUT LIVING WITH OPP SEX PARTNER 
#3 WIDOWED 
#4 DIVORCED 
#5 SEPARATED FOR REASONS OF MARITAL DISCORD 
#6 NEVER BEEN MARRIED 

# This person has a 75% chance of being currently married, 
# a 13% chance of being "not married but living with opposite sex partner.


,0,1,2,3,4,5
0,0.748384,0.125474,0.001103,0.035295,0.023813,0.065931
